In [1]:
import dask
from dask_kubernetes import KubeCluster
import numpy as np

In [2]:
# Specify a remote deployment using a load blanacer
dask.config.set({"kubernetes.scheduler-service-type": "LoadBalancer"})

In [3]:
cluster = KubeCluster.from_yaml('worker-spec.yaml', namespace='dask', deploy_mode='remote')

Creating scheduler pod on cluster. This may take some time.


In [4]:
cluster.adapt(minimum=1, maximum=10)

distributed.deploy.adaptive - INFO - Adaptive scaling started: minimum=1 maximum=10


In [5]:
# Example usage
from dask.distributed import Client
import dask.array as da

# Connect Dask to the cluster
client = Client(cluster)
client # the repr gives us useful links

Client Scheduler: tcp://192.168.3.119:8786 Dashboard: http://192.168.3.119:8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [6]:
client.scheduler_comm.comm.handshake_info()

{'compression': 'lz4', 'python': (3, 8, 0), 'pickle-protocol': 5}

In [7]:
# Create a large array and calculate the mean
array = da.ones((1000, 1000, 1000))
print(array.mean().compute())  # Should print 1.0|

1.0


So now we know the cluster is doing ok :)

Configure dask to talk to our local MinIO

In [55]:
minio_storage_options = {
#    "anon": "false",
    "key": "YOURACCESSKEY",
    "secret": "YOURSECRETKEY",
    "client_kwargs": {
        "endpoint_url": "http://minio-1602984784.minio.svc.cluster.local:9000",
        "region_name": 'us-east-1'
    },
    "config_kwargs": {"s3": {"signature_version": 's3v4'}},
}

Download the GH archive data

In [9]:
import datetime
import dask.dataframe as dd

In [10]:
current_date=datetime.datetime(2020,10,1, 1)

In [11]:
gh_archive_files=[]

In [12]:
while current_date < datetime.datetime.now() -  datetime.timedelta(days=1):
    current_date = current_date + datetime.timedelta(hours=1)
    datestring = f'{current_date.year}-{current_date.month:02}-{current_date.day:02}-{current_date.hour}'
    gh_url = f'http://data.githubarchive.org/{datestring}.json.gz'
    gh_archive_files.append(gh_url)

In [13]:
df = dd.read_json(gh_archive_files[0], compression='gzip')

distributed.deploy.adaptive - INFO - Retiring workers [1, 2, 3, 4, 5, 6]


In [14]:
len(df)

92084

In [15]:
from fsspec.registry import known_implementations
known_implementations

{'file': {'class': 'fsspec.implementations.local.LocalFileSystem'},
 'memory': {'class': 'fsspec.implementations.memory.MemoryFileSystem'},
 'dropbox': {'class': 'dropboxdrivefs.DropboxDriveFileSystem',
  'err': 'DropboxFileSystem requires "dropboxdrivefs","requests" and "dropbox" to be installed'},
 'http': {'class': 'fsspec.implementations.http.HTTPFileSystem',
  'err': 'HTTPFileSystem requires "requests" and "aiohttp" to be installed'},
 'https': {'class': 'fsspec.implementations.http.HTTPFileSystem',
  'err': 'HTTPFileSystem requires "requests" and "aiohttp" to be installed'},
 'zip': {'class': 'fsspec.implementations.zip.ZipFileSystem'},
 'gcs': {'class': 'gcsfs.GCSFileSystem',
  'err': 'Please install gcsfs to access Google Storage'},
 'gs': {'class': 'gcsfs.GCSFileSystem',
  'err': 'Please install gcsfs to access Google Storage'},
 'gdrive': {'class': 'gdrivefs.GoogleDriveFileSystem',
  'err': 'Please install gdrivefs for access to Google Drive'},
 'sftp': {'class': 'fsspec.impl

In [25]:
df.columns

Index(['id', 'type', 'actor', 'repo', 'payload', 'public', 'created_at',
       'org'],
      dtype='object')

In [57]:
df.to_csv("s3://dask-test/boop-test", storage_options=minio_storage_options)

['dask-test/boop-test/0.part']

In [ ]:
counter.value().result()

In [ ]:
counter.value().result()

In [ ]:
counter.value().result()

In [ ]:
f[0]

In [ ]:
# Create a large array and calculate the mean
array = da.ones((1000, 1000, 1000))
print(array.mean().compute())  # Should print 1.0|

In [37]:
import boto

In [42]:
import boto3
from botocore.client import Config


s3 = boto3.resource('s3',
                    endpoint_url='http://localhost:9000',
                    aws_access_key_id='YOURACCESSKEY',
                    aws_secret_access_key='YOURSECRETKEY',
                    config=Config(signature_version='s3v4'),
                    region_name='us-east-1')


In [52]:
s3.Bucket('dask-test').upload_file('/etc/lsb-release','lsb-release')